In [1]:
import keras
from keras.models import Model, Sequential
import numpy as np
from keras.optimizers import SGD, Adam
from keras.layers import Dense, LSTM, Input, Reshape, merge

Using TensorFlow backend.


# 1. Loading input data and testing data

In [2]:
X = np.array([[0.2 , 0.3, 0.4, 0.5], [0.3, 0.4, 0.5, 0.6], [0.2, 0.2, 0.3, 0.3]])
Y = np.array([[0.3, 0.4, 0.5, 0.6], [0.4, 0.5, 0.6, 0.7], [0.3, 0.3, 0.3, 0.3]])
noise = np.random.random((3, 4)) 

In [3]:
new_data = np.array([0.5, 0.2, 0.3, 0.5])
new_data = np.reshape(new_data, (1,4))

# 2. Defining the generator and the discriminator

In [4]:
# try out different numbers of output dimensions
def get_generative_model(input_dim = 4, num_recurrent_units = 1):    
    model = Sequential()
    model.add(Dense(1, activation='relu', input_dim = 4))
    predicted = model.add(Dense(4, activation='sigmoid'))
    return model 

In [49]:
generator = get_generative_model()

for i in range(3):
    weights = generator.layers[0].get_weights()[0]
    biases = generator.layers[0].get_weights()[1]
    print weights
    print "biases"
    print biases

    generator.compile(loss='binary_crossentropy', optimizer='rmsprop')
    predictions = generator.predict(X)
    print predictions
    generator.fit(X, Y)
    weights2 = generator.get_weights()
    print weights2

#generator.predict(Xwithnoise)

[[ 0.37064004]
 [-0.8000145 ]
 [ 0.82944107]
 [ 1.02208614]]
biases
[ 0.]
[[ 0.53198922  0.35238305  0.52333963  0.47842318]
 [ 0.53868514  0.32378441  0.52823347  0.47389776]
 [ 0.52220607  0.39599964  0.51619643  0.48502773]]
Epoch 1/10
3/3 [==============================] - 0s - loss: 0.6997
Epoch 2/10
3/3 [==============================] - 0s - loss: 0.6991
Epoch 3/10
3/3 [==============================] - 0s - loss: 0.6986
Epoch 4/10
3/3 [==============================] - 0s - loss: 0.6982
Epoch 5/10
3/3 [==============================] - 0s - loss: 0.6979
Epoch 6/10
3/3 [==============================] - 0s - loss: 0.6976
Epoch 7/10
3/3 [==============================] - 0s - loss: 0.6973
Epoch 8/10
3/3 [==============================] - 0s - loss: 0.6971
Epoch 9/10
3/3 [==============================] - 0s - loss: 0.6969
Epoch 10/10
3/3 [==============================] - 0s - loss: 0.6966
[array([[ 0.35400149],
       [-0.81670916],
       [ 0.81278151],
       [ 1.00539446]], d

In [24]:
weights

[array([[-0.21257015, -0.39053315, -0.39324281],
        [ 0.58025825,  0.15116811, -0.88164705],
        [ 0.66209418, -0.39955997, -0.14333114],
        [-0.40137407, -0.56507242,  0.90970021]], dtype=float32),
 array([-0.00760559,  0.        ,  0.00154553], dtype=float32),
 array([[ 0.84257764,  0.76163155,  0.48641929,  0.00907715],
        [ 0.19483352, -0.68577266, -0.69406331,  0.44337165],
        [-0.43028358, -0.90869778,  0.48243687, -0.50956267]], dtype=float32),
 array([-0.00515406, -0.00333362, -0.00149658,  0.00088152], dtype=float32)]

In [5]:
def get_discriminative_model():
    model = Sequential()
    model.add(Dense(12, input_dim=4, init='uniform', activation='relu'))
    model.add(Dense(1, init='uniform', activation='sigmoid'))
    return model

# 3. Running through model to get predictions from the generator

In [6]:
generator = get_generative_model()
generator.compile(loss='binary_crossentropy', optimizer='sgd')
generator.fit(X, Y)
generated_values = generator.predict(X)

Epoch 1/10
3/3 [==============================] - 0s - loss: 0.6931
Epoch 2/10
3/3 [==============================] - 0s - loss: 0.6931
Epoch 3/10
3/3 [==============================] - 0s - loss: 0.6931
Epoch 4/10
3/3 [==============================] - 0s - loss: 0.6931
Epoch 5/10
3/3 [==============================] - 0s - loss: 0.6930
Epoch 6/10
3/3 [==============================] - 0s - loss: 0.6930
Epoch 7/10
3/3 [==============================] - 0s - loss: 0.6930
Epoch 8/10
3/3 [==============================] - 0s - loss: 0.6930
Epoch 9/10
3/3 [==============================] - 0s - loss: 0.6929
Epoch 10/10
3/3 [==============================] - 0s - loss: 0.6929


In [7]:
generated_values

array([[ 0.49896124,  0.49937671,  0.49979225,  0.50020772],
       [ 0.49896124,  0.49937671,  0.49979225,  0.50020772],
       [ 0.49896124,  0.49937671,  0.49979225,  0.50020772]], dtype=float32)

In [8]:
X

array([[ 0.2,  0.3,  0.4,  0.5],
       [ 0.3,  0.4,  0.5,  0.6],
       [ 0.2,  0.2,  0.3,  0.3]])

# 4. Creating input for the discriminator

In [9]:
# X = np.reshape(X, (3,4))
# Y = np.reshape(Y, (3,4))
# generated_input_to_discriminator = np.concatenate([X, generated_values], axis = 1)
# generated_input_to_discriminator = np.reshape(generated_input_to_discriminator,(3,8))
generated_input_to_discriminator = np.concatenate([X, generated_values], axis = 1) 
true_input_to_discriminator = np.concatenate([X, Y], axis = 1)

#true_input_to_discriminator = np.reshape(true_input_to_discriminator,(3,8))
#true_input_to_discriminator = Y

Xdisc = np.concatenate((generated_input_to_discriminator, true_input_to_discriminator))
Ydisc = [1] * len(generated_input_to_discriminator) + [0] * len(true_input_to_discriminator)
new_data_disc = np.array([0.2,0.5,0.4, 0.2, 0.3, 0.6, 0.5, 0.3])
#new_data_disc = np.reshape (new_data_disc, (2,8))

In [10]:
Xdisc = Xdisc.tolist()

# 5. Running through model to get output from the discriminator

In [11]:
discriminator = get_discriminative_model()
discriminator.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
discriminator.fit (Xdisc, Ydisc)
discriminator.predict(Xdisc)

ValueError: Error when checking model input: expected dense_input_2 to have shape (None, 4) but got array with shape (6, 8)

# 6. Connecting the discriminator to the generator

In [24]:
def combined_model(generator, discriminator):
    model = Sequential()
    model.add(generator)
    # Here generator's output are arrays that are 8 numbers long
    # Discriminator takes inpt that is 16 numbers long
    discriminator.trainable = False
    model.add(discriminator)
    return model

In [25]:
generator_containing_disciminator = combined_model(generator, discriminator)

# ALTERNATIVE 
DISCRIMINATOR THAT TAKES THE GENERATORS OUTPUT IMMEDIATELY

In [281]:
def get_generative_model(hidden_neurons = 12, input_dim=4):    
    model = Sequential()
    model.add(Dense(hidden_neurons, activation='relu', input_dim = 4))
    model.add(Dense(4, activation='sigmoid'))
    #model.add(Reshape((4,1), input_shape=(1,4)))
    return model 

In [272]:
def get_discriminative_model():
    model = Sequential()
    model.add(Dense(12, input_dim=4, init='uniform', activation='relu'))
    model.add(Dense(1, init='uniform', activation='sigmoid'))
    return model

In [309]:
new_data = np.reshape(new_data, (1,4))

In [311]:
generator = get_generative_model()
generated_values = generator.predict(new_data)
new_data.shape

(1, 4)

In [285]:
real_input = np.reshape(Y, (1,4))
Xdisc = np.concatenate((generated_values, real_input))
Ydisc = [1] * len(generated_values) + [0] * len(real_input)
new_data = np.array([0.6, 0.6, 0.6, 0.6])
new_data = np.reshape(new_data, (1,4))

In [287]:
discriminator = get_discriminative_model()
discriminator.compile(loss='binary_crossentropy', optimizer='sgd')
discriminator.fit (Xdisc, Ydisc)
discriminator.predict(new_data)

Epoch 1/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 2/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 3/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 4/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 5/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 6/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 7/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 8/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 9/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 10/10
2/2 [==============================] - 0s - loss: 0.6932


array([[ 0.49939361]], dtype=float32)

## Combining the discriminator with the generator

In [291]:
def combined_model(generator, discriminator):
    model = Sequential()
    model.add(generator)
    
    discriminator.trainable = False
    model.add(discriminator)
    return model

In [292]:
generator_containing_disciminator = combined_model(generator, discriminator)

In [356]:
Ydisc1 = [1]

In [358]:
generator_containing_disciminator.fit(X, Ydisc1)

AttributeError: 'int' object has no attribute 'shape'

In [295]:
# # 8. Setting optimizers for the generator and the discriminator
d_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
g_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
generator.compile(loss='binary_crossentropy', optimizer='sgd')
generator_containing_disciminator.compile(loss='binary_crossentropy', optimizer=g_optim)
discriminator.trainable = True
discriminator.compile(loss='binary_crossentropy', optimizer=d_optim)

# Getting losses

In [353]:
def get_losses(i):
    epoch = 0
    print ('Epoch:', epoch+1)
    d_losses = []
    g_losses = []
    for batchno in range(3):
        inputdata = X
        generated_audio = generator.predict(inputdata)
        real_audio = Y
        Xdisc = np.concatenate((real_audio, generated_audio))
        Ydisc = [1] * len(real_audio) + [0] * len(generated_audio)
        d_loss = discriminator.fit(Xdisc, Ydisc)  # later change to train on batch
        d_losses.append(d_loss)
        discriminator.trainable = False
        g_loss = generator_containing_disciminator.fit(inputdata, len(real_audio))
        #print g_loss
    epoch =+ 1
    return True

In [352]:
get_losses(0)

('Epoch:', 1)
Epoch 1/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 2/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 3/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 4/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 5/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 6/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 7/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 8/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 9/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 10/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 1/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 2/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 3/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 4/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 5/10
2/2 [=================

True

In [ ]:

        g_loss = generator_containing_disciminator.train_on_batch(inputdata, [1]*batch_size)
        # take into account example data also. input audio - example of laugter - paper about images from Daniel. 
        
        # 9.8 Calculate losses for the generator
        g_losses.append(g_loss)
        
        # I thought the discriminator was set to not trainable already in step 9.7??
        discriminator.trainable = True
        # Print how many batches have been trained on 
        sys.stdout.write(' + batch: ' + str(index+1) + '/' + str(n_batches) + '\r')
        # Everything from the buffer will be written in the terminal 
        sys.stdout.flush()
    return d_losses, g_losses
